In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import torch
import transformers
import inspect
import time
import logging

from tqdm import trange, tqdm, tqdm_notebook, tqdm_pandas, tqdm_gui
from datetime import datetime
from tqdm import tqdm
from transformers import BertConfig, BertModel, BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_constant_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

'GeForce RTX 2080 Ti'

# Load data & pre-processing

In [2]:
def preprocessing(df):
    """
    Preprocessing step
    As above dataframe heads, there is a lot of <br /> character 
    """
    df.sentence = df.sentence.str.replace('<br />','')
    df.sentence = df.sentence.str.lower()
    return df

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

train = preprocessing(train)
test = preprocessing(test)

print('Train data:\n{}\n\nTest data:\n{}'.format(train.head(5), test.head(5)))

Train data:
                                            sentence  sentiment  polarity
0  this is a very bland and inert production of o...          2         0
1  i've seen this film in avant-premiere at imagi...          7         1
2  revolt of the zombies (2 outta 5 stars) no, th...          4         0
3  may contain minor spoilers.dressed to kill, ha...          7         1
4  (spoilers)i shoulda figured. the dvd didn't ev...          2         0

Test data:
                                            sentence  sentiment  polarity
0  i loved this movie so much. i'm a big fan of a...         10         1
1  the stark, cold landscape of big sky country, ...          9         1
2  this cheapo exploitation flick is some genuine...          2         0
3  this movie has been promoting in everywhere in...          1         0
4  this is a great off-the-wall romantic comedy a...          8         1


# Tokenization & Create inputs for model

In [4]:
class BertModelBonz():
    def __init__(self, load_model=None, load_config=None, model='bert-base-uncased', max_len=512, batch_size=6):
        self.pre_trained_model = model
        self.max_len = max_len
        self.batch_size = batch_size
        # Setting model
        if load_model is not None:
            self.model = torch.load(load_model)
        elif load_config is not None:
            self.model = BertForSequenceClassification(load_config)
            self.max_len = load_config.max_position_embeddings
        else:    
            self.model = BertForSequenceClassification.from_pretrained(self.pre_trained_model)
        self.tokenizer = BertTokenizer.from_pretrained(self.pre_trained_model)
        self.tokenizer.max_len = max_len
        self.optimizer = AdamW(params = self.model.parameters(), lr=1e-5)
    
    def create_ids(self, sentences):
        logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR) #Disable tokenizer logs, it's really annoy
        input_ids = []
        for sen in tqdm_notebook(sentences, desc="Create Ids"):
            tmp = self.tokenizer.encode(sen)
            input_ids.append(tmp)
        input_ids = pad_sequences(input_ids, 
                                  maxlen=self.max_len, 
                                  dtype='int64', 
                                  truncating='post', 
                                  padding='post')
        return input_ids
    
    def prepare_data(self, input_ids, input_labels=None):
        input_ids = torch.tensor(self.create_ids(input_ids))
        if input_labels is None:
            return DataLoader(TensorDataset(input_ids), 
                              batch_size=self.batch_size)
        else:
            input_labels = torch.tensor(input_labels)
            return DataLoader(TensorDataset(input_ids, input_labels), 
                              batch_size=self.batch_size)
        
    def flat_accuracy(self, preds, labels):
        pred_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    def train(self, dataloader, epochs=4):
        self.train_loss_set =[]
        for i in trange(epochs, desc="Epoch"):
            # Training model
            self.model.to(device)
            self.model.train()
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            for input_ids, input_labels in tqdm_notebook(dataloader):
                self.optimizer.zero_grad()
                loss = self.model(input_ids=input_ids.cuda(), labels=input_labels.cuda())[0]
                self.train_loss_set.append(loss)    
                loss.backward()
                self.optimizer.step()
                
                tr_loss += loss.item()
                nb_tr_examples += input_ids.size(0)
                nb_tr_steps += 1
            print("Train loss: {}".format(tr_loss/nb_tr_steps))

            # Evaluation
            self.model.eval()
            eval_loss, eval_accuracy = 0, 0
            nb_eval_steps, nb_eval_examples = 0, 0
            for input_ids, input_labels in dataloader:
                with torch.no_grad():
                    logits = self.model(input_ids.cuda())[0]
                logits = logits.detach().cpu().numpy()
                label_ids = input_labels.to('cpu').numpy()
                tmp_eval_accuracy = self.flat_accuracy(logits, label_ids)
                eval_accuracy += tmp_eval_accuracy
                nb_eval_steps += 1
            print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
            
            #Save model for each epoch:
            filename = 'bert_512_epoch'+str(i)+'.sd'
            filepath = './model/bert/'+filename
            torch.save(self.model.state_dict(), filepath)
            
    def predict(self, test_data, test_labels):
        test_ids = self.create_ids(test_data)
        test_inputs = torch.tensor(test_ids)
        test_dataloader = DataLoader(test_inputs, batch_size=self.batch_size)
        
        # Preditcion
        self.model.to(device)
        self.model.eval()
        self.predictions = []
        for input_ids in tqdm_notebook(test_dataloader, desc="Predicting"):
            with torch.no_grad():
                logits = self.model(input_ids.to(device))[0]
            logits = logits.detach().cpu().numpy()
            self.predictions.append(logits)
        self.predictions = [j for i in self.predictions for j in i]
        self.predictions = np.argmax(self.predictions, axis=1)
        print(classification_report(self.predictions, test_labels, digits=4))



In [5]:
bert_model = BertModelBonz(batch_size=6)


#Train model

train.dataloader = bert_model.prepare_data(input_ids=train['sentence'], input_labels=train['polarity'])
bert_model.train(train.dataloader)
#torch.save(bert_model.model, 'bert_eb1024_1e5_e4.pth')




C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
Epoch:   0%|                                                                                     | 0/4 [00:00<?, ?it/s]


Train loss: 0.25542188695409984
Validation Accuracy: 0.9586233101351972


Epoch:  25%|██████████████████▎                                                      | 1/4 [31:02<1:33:07, 1862.54s/it]


Train loss: 0.12138432963141214
Validation Accuracy: 0.9831213502919798


Epoch:  50%|███████████████████████████████████▌                                   | 2/4 [1:02:00<1:02:02, 1861.26s/it]


Train loss: 0.0652414504841357
Validation Accuracy: 0.987840972722185


Epoch:  75%|██████████████████████████████████████████████████████▊                  | 3/4 [1:33:04<31:02, 1862.00s/it]


Train loss: 0.04449292355971937
Validation Accuracy: 0.9949204063674922


Epoch: 100%|█████████████████████████████████████████████████████████████████████████| 4/4 [2:04:08<00:00, 1862.48s/it]


'\n#Predict model\nbert_model.predict(test_data=test.sentence, test_labels=test.polarity)\n'

In [6]:

#Predict model
bert_model.predict(test_data=test.sentence, test_labels=test.polarity)



              precision    recall  f1-score   support

           0     0.9211    0.9444    0.9326     12192
           1     0.9458    0.9230    0.9343     12808

   micro avg     0.9334    0.9334    0.9334     25000
   macro avg     0.9334    0.9337    0.9334     25000
weighted avg     0.9337    0.9334    0.9335     25000



In [8]:
for i in range(4):
    filename = 'bert_512_epoch'+str(i)+'.sd'
    filepath = './model/bert/'+filename
    bert_model.model.load_state_dict(torch.load(filepath))
    bert_model.model.eval()
    print('Model ',filename,':')
    bert_model.predict(test_data=test.sentence, test_labels=test.polarity)

Model  bert_512_epoch0.sd :



              precision    recall  f1-score   support

           0     0.9084    0.9448    0.9262     12019
           1     0.9469    0.9118    0.9290     12981

   micro avg     0.9276    0.9276    0.9276     25000
   macro avg     0.9276    0.9283    0.9276     25000
weighted avg     0.9284    0.9276    0.9277     25000

Model  bert_512_epoch1.sd :



              precision    recall  f1-score   support

           0     0.9376    0.9228    0.9301     12701
           1     0.9215    0.9366    0.9290     12299

   micro avg     0.9296    0.9296    0.9296     25000
   macro avg     0.9296    0.9297    0.9296     25000
weighted avg     0.9297    0.9296    0.9296     25000

Model  bert_512_epoch2.sd :



              precision    recall  f1-score   support

           0     0.9002    0.9581    0.9282     11744
           1     0.9606    0.9059    0.9324     13256

   micro avg     0.9304    0.9304    0.9304     25000
   macro avg     0.9304    0.9320    0.9303     25000
weighted avg     0.9322    0.9304    0.9305     25000

Model  bert_512_epoch3.sd :



              precision    recall  f1-score   support

           0     0.9211    0.9444    0.9326     12192
           1     0.9458    0.9230    0.9343     12808

   micro avg     0.9334    0.9334    0.9334     25000
   macro avg     0.9334    0.9337    0.9334     25000
weighted avg     0.9337    0.9334    0.9335     25000

